In [7]:
import math

In [2]:
!pip install osmnx

  Using cached shapely-2.1.2-cp312-cp312-win_amd64.whl.metadata (7.1 kB)
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   -- ------------------------------------- 1.0/19.2 MB 8.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.9/19.2 MB 8.4 MB/s eta 0:00:02
   ----------- ---------------------------- 5.5/19.2 MB 9.1 MB/s eta 0:00:02
   -------------- ------------------------- 6.8/19.2 MB 8.4 MB/s eta 0:00:02
   ---------------- ----------------------- 8.1/19.2 MB 8.2 MB/s eta 0:00:02
   -------------------- ------------------- 10.0/19.2 MB 8.3 MB/s eta 0:00:02
   ----------------------- ---------------- 11.3/19.2 MB 7.8 MB/s eta 0:00:02
   ------------------------- -------------- 12.3/19.2 MB 7.5 MB/s eta 0:00:01
   ---------------------------- ----------- 13.9/19.2 MB 7.5 MB/s eta 0:00:01
   -------------------------------- ------- 15.7/19.2 MB 7.6 MB/s eta 0:00:01
   ------------------------------------- -- 17.8/19.2 MB 7.8 MB/s eta 0:00:01
   -

# Data of a Square Area

In [ ]:
import googlemaps
import pandas as pd
import numpy as np
from datetime import datetime
import osmnx as ox
import random

# --- Configuration ---
API_KEY = "Add your google traffic api key"

# Define the area and grid size as before
AREA_BOUNDS = {
    "sw": (28.542092, 77.025868),  # South-West corner (lat, lon)
    "ne": (28.741011, 77.307083)  # North-East corner
}
GRID_SIZE_X = 5
GRID_SIZE_Y = 5
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Main Script ---
gmaps = googlemaps.Client(key=API_KEY)

def generate_grid_points(bounds, num_x, num_y):
    lats = np.linspace(bounds["sw"][0], bounds["ne"][0], num_y)
    lons = np.linspace(bounds["sw"][1], bounds["ne"][1], num_x)
    return [(lat, lon) for lat in lats for lon in lons]

grid_points = generate_grid_points(AREA_BOUNDS, GRID_SIZE_X, GRID_SIZE_Y)

all_path_points = []
total_paths = (GRID_SIZE_X * GRID_SIZE_Y) ** 2 - (GRID_SIZE_X * GRID_SIZE_Y)
path_counter = 0


for i, origin in enumerate(grid_points):
    for j, destination in enumerate(grid_points):
        if i == j:
            continue
        
        path_counter += 1
        print(f"  Processing path {path_counter}/{total_paths} (Point {i} -> Point {j})")

     
        # Request directions for this specific pair
        directions_result = gmaps.directions(origin,
                                                destination,
                                                mode="driving",
                                                departure_time=datetime.now())

        if not directions_result:
            continue

        # --- Extract Key Information ---
        route = directions_result[0] # Use the primary recommended route
        leg = route['legs'][0]
        
        # Calculate congestion factor for this specific path
        duration_val = leg['duration']['value']
        duration_traffic_val = leg.get('duration_in_traffic', {}).get('value', duration_val)
        congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

        # --- Decode the Polyline to get actual road coordinates ---
        encoded_polyline = route['overview_polyline']['points']
        decoded_path = googlemaps.convert.decode_polyline(encoded_polyline)

        # Create a unique ID for this path
        path_id = f"Point {i} -> Point {j}"
        
        # For each coordinate in the decoded path, create a row for Tableau
        for order, point in enumerate(decoded_path):
            all_path_points.append({
                'Path_ID': path_id,
                'Point_Order': order + 1,
                'Latitude': point['lat'],
                'Longitude': point['lng'],
                'Congestion_Factor': round(congestion_factor, 2),
                'Distance_Text': leg['distance']['text'],
                'Duration_in_Traffic_Text': leg.get('duration_in_traffic', {}).get('text', 'N/A')
            })




# Convert the final list into a DataFrame
tableau_df = pd.DataFrame(all_path_points)

# Save the detailed path data to a new CSV file
tableau_df.to_csv(OUTPUT_FILENAME, index=False)

print(f"\n Successfully generated detailed path data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")

# --- Configuration ---
API_KEY = "Add your google traffic api key"
API_KEY = "Enter API key here "

# Define the area and grid size as before
AREA_BOUNDS = {
    "sw": (28.341944, 76.811111),  # South-West corner (lat, lon)
    "ne": (28.928889, 77.473056)   # North-East corner
}
GRID_SIZE_X = 5
GRID_SIZE_Y = 5
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Main Script ---
gmaps = googlemaps.Client(key=API_KEY)

def generate_grid_points(bounds, num_x, num_y):
    lats = np.linspace(bounds["sw"][0], bounds["ne"][0], num_y)
    lons = np.linspace(bounds["sw"][1], bounds["ne"][1], num_x)
    return [(lat, lon) for lat in lats for lon in lons]

grid_points = generate_grid_points(AREA_BOUNDS, GRID_SIZE_X, GRID_SIZE_Y)

all_path_points = []
total_paths = (GRID_SIZE_X * GRID_SIZE_Y) ** 2 - (GRID_SIZE_X * GRID_SIZE_Y)
path_counter = 0


for i, origin in enumerate(grid_points):
    for j, destination in enumerate(grid_points):
        if i == j:
            continue
        
        path_counter += 1
        print(f"  Processing path {path_counter}/{total_paths} (Point {i} -> Point {j})")

     
        # Request directions for this specific pair
        directions_result = gmaps.directions(origin,
                                                destination,
                                                mode="driving",
                                                departure_time=datetime.now())

        if not directions_result:
            continue

        # --- Extract Key Information ---
        route = directions_result[0] # Use the primary recommended route
        leg = route['legs'][0]
        
        # Calculate congestion factor for this specific path
        duration_val = leg['duration']['value']
        duration_traffic_val = leg.get('duration_in_traffic', {}).get('value', duration_val)
        congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

        # --- Decode the Polyline to get actual road coordinates ---
        encoded_polyline = route['overview_polyline']['points']
        decoded_path = googlemaps.convert.decode_polyline(encoded_polyline)

        # Create a unique ID for this path
        path_id = f"Point {i} -> Point {j}"
        
        # For each coordinate in the decoded path, create a row for Tableau
        for order, point in enumerate(decoded_path):
            all_path_points.append({
                'Path_ID': path_id,
                'Point_Order': order + 1,
                'Latitude': point['lat'],
                'Longitude': point['lng'],
                'Congestion_Factor': round(congestion_factor, 2),
                'Distance_Text': leg['distance']['text'],
                'Duration_in_Traffic_Text': leg.get('duration_in_traffic', {}).get('text', 'N/A')
            })




# Convert the final list into a DataFrame
tableau_df = pd.DataFrame(all_path_points)

# Save the detailed path data to a new CSV file
tableau_df.to_csv(OUTPUT_FILENAME, index=False)

print(f"\n Successfully generated detailed path data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")

c:\conda\envs\pyenv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


  Processing path 1/600 (Point 0 -> Point 1)
  Processing path 2/600 (Point 0 -> Point 2)
  Processing path 3/600 (Point 0 -> Point 3)
  Processing path 4/600 (Point 0 -> Point 4)
  Processing path 5/600 (Point 0 -> Point 5)
  Processing path 6/600 (Point 0 -> Point 6)
  Processing path 7/600 (Point 0 -> Point 7)
  Processing path 8/600 (Point 0 -> Point 8)
  Processing path 9/600 (Point 0 -> Point 9)
  Processing path 10/600 (Point 0 -> Point 10)
  Processing path 11/600 (Point 0 -> Point 11)
  Processing path 12/600 (Point 0 -> Point 12)
  Processing path 13/600 (Point 0 -> Point 13)
  Processing path 14/600 (Point 0 -> Point 14)
  Processing path 15/600 (Point 0 -> Point 15)
  Processing path 16/600 (Point 0 -> Point 16)
  Processing path 17/600 (Point 0 -> Point 17)
  Processing path 18/600 (Point 0 -> Point 18)
  Processing path 19/600 (Point 0 -> Point 19)
  Processing path 20/600 (Point 0 -> Point 20)
  Processing path 21/600 (Point 0 -> Point 21)
  Processing path 22/600 (Point

In [3]:
tableau_df

,Path_ID,Point_Order,Latitude,Longitude,Congestion_Factor,Distance_Text,Duration_in_Traffic_Text
0,Point 0 -> Point 1,1,28.48000,76.93017,1.00,3.0 km,9 mins
1,Point 0 -> Point 1,2,28.47953,76.93015,1.00,3.0 km,9 mins
2,Point 0 -> Point 1,3,28.47958,76.93033,1.00,3.0 km,9 mins
3,Point 0 -> Point 1,4,28.47974,76.93075,1.00,3.0 km,9 mins
4,Point 0 -> Point 1,5,28.47982,76.93090,1.00,3.0 km,9 mins
...,...,...,...,...,...,...,...
8848,Point 8 -> Point 7,49,28.52003,76.94858,1.11,3.2 km,9 mins
8849,Point 8 -> Point 7,50,28.51917,76.94813,1.11,3.2 km,9 mins
8850,Point 8 -> Point 7,51,28.51917,76.94864,1.11,3.2 km,9 mins
8851,Point 8 -> Point 7,52,28.51917,76.94953,1.11,3.2 km,9 mins


# Data for Delhi

In [ ]:
import pandas as pd
from datetime import datetime
import osmnx as ox
import networkx as nx
import random
import math
import requests        
import urllib.parse    

# --- Configuration ---

API_KEY = "use you api" # change it with your TomTom API key



PLACE_LIST = [
    "Delhi, India",
    "Gurugram, India",
    "Noida, India",
    "Ghaziabad, India",
    "Faridabad, India"
]

COMBINED_PLACE_LIST = "Mumbai"

POINTS_PER_ROAD_TYPE = 20 # Sample size per Road Type

RANDOM_STATE = 77

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{COMBINED_PLACE_LIST}_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Define the road types to process individually ---
ROAD_TYPES_TO_PROCESS = ['motorway', 'trunk', 'primary', 'secondary','tertiary']

# --- Helper functions to format TomTom's output (meters/seconds) ---

def format_duration(seconds):
    """Converts seconds into a human-readable 'X hours Y mins' or 'Y mins' string."""
    if seconds is None:
        return "N/A"
    minutes = math.ceil(seconds / 60)
    if minutes < 60:
        return f"{minutes} mins"
    else:
        hours = minutes // 60
        rem_minutes = minutes % 60
        return f"{hours} hours {rem_minutes} mins"

def format_distance(meters):
    """Converts meters into a human-readable 'X.Y km' string."""
    if meters is None:
        return "N/A"
    km = round(meters / 1000, 1)
    return f"{km} km"

# --- No TomTom Client Initialization needed ---

# This list will hold data from all road type loops
all_path_points_data = []

print(f"Starting traffic analysis for {PLACE_LIST}...")
print("-" * 50)

# --- Main loop to process each road type separately ---
for road_type in ROAD_TYPES_TO_PROCESS:
    print(f"\nProcessing road type: '{road_type.upper()}'")

    graphs_for_this_road_type = []
    print(f"  Downloading '{road_type}' network from OpenStreetMap for {len(PLACE_LIST)} places...")
    for place in PLACE_LIST:
        try:
            # Fetch network for ONLY the current road type
            road_filter = f'["highway"~"{road_type}"]'
            G_place = ox.graph_from_place(place, network_type='drive', custom_filter=road_filter)
            
            if G_place.nodes:
                graphs_for_this_road_type.append(G_place)
                print(f"    - Successfully fetched '{road_type}' for {place}")
            else:
                print(f"    - No '{road_type}' roads found in {place}.")
                
        except Exception as e:
            print(f"    - Error fetching data for {place}: {e}")
    
    if not graphs_for_this_road_type:
        print(f"  - No '{road_type}' roads found in any specified location. Skipping.")
        continue
        
    print(f"  Combining {len(graphs_for_this_road_type)} graphs for '{road_type}'...")
    G = nx.compose_all(graphs_for_this_road_type)

    try:
        if not G.nodes:
            print(f"  - Combined graph for '{road_type}' is empty. Skipping.")
            continue

        # Get all points for this specific network
        nodes = list(G.nodes)
        all_road_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in nodes]
        print(f"  Found {len(all_road_points)} total points for this road type.")

        if len(all_road_points) < 2:
            print(f"  - Not enough points ({len(all_road_points)}) to create paths. Skipping.")
            continue

        # Take a random sample of these points
        random.seed(RANDOM_STATE)
        sample_size = min(POINTS_PER_ROAD_TYPE, len(all_road_points))
        sampled_points = random.sample(all_road_points, sample_size)
        print(f"  Using a random sample of {len(sampled_points)} points.")

        # Generate paths and get traffic data for this sample
        total_paths = len(sampled_points) * (len(sampled_points) - 1)
        path_counter = 0

        if total_paths == 0:
            print("  - Only one point sampled, cannot create paths. Skipping.")
            continue

        print(f"  Starting TomTom API calls for {total_paths} paths...")
        for i, origin in enumerate(sampled_points):
            for j, destination in enumerate(sampled_points):
                if i == j:
                    continue

                path_counter += 1
                
                # --- TomTom API Call using 'requests' ---
                try:
                    origin_str = f"{origin[0]},{origin[1]}"
                    dest_str = f"{destination[0]},{destination[1]}"
                    
                    locations = urllib.parse.quote(f"{origin_str}:{dest_str}")
                    
                    depart_at_iso = datetime.now().isoformat()

                    base_url = f"https://api.tomtom.com/routing/1/calculateRoute/{locations}/json"
                    
                    params = {
                        'key': API_KEY,
                        'departAt': depart_at_iso,
                        'traffic': 'true',
                        'computeTravelTimeFor': 'all', # Request both traffic and no-traffic times
                        'travelMode': 'car',
                        'routeType': 'fastest'
                    }

                    # Make the GET request
                    response = requests.get(base_url, params=params)
                    response.raise_for_status() # Raise an exception for bad status codes (4xx, 5xx)
                    
                    directions_result = response.json()


                except requests.exceptions.RequestException as e:
                    print(f"      - TomTom API Error for path {i}-to-{j}: {e}")
                    continue 

                if not directions_result or 'routes' not in directions_result or not directions_result['routes']:
                    print(f"      - No route found by TomTom for path {i}-to-{j}.")
                    continue
                # --- End TomTom API Call ---

                # --- Extract TomTom Info ---
                route = directions_result['routes'][0]
                summary = route['summary']

                duration_val = summary.get('noTrafficTravelTimeInSeconds')
                duration_traffic_val = summary.get('travelTimeInSeconds')
                distance_val_meters = summary.get('lengthInMeters')
                
                # Handle cases where one value might be missing
                if duration_val is None:
                    duration_val = duration_traffic_val
                if duration_traffic_val is None:
                    duration_traffic_val = duration_val
                if duration_val is None or duration_traffic_val is None:
                    print(f"      - Missing duration data for path {i}-to-{j}. Skipping.")
                    continue
                
                # Calculate congestion
                congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

                leg = route['legs'][0]
                decoded_path = leg['points'] 

                # Get text representations using our helper functions
                distance_text = format_distance(distance_val_meters)
                duration_traffic_text = format_duration(duration_traffic_val)
                
                path_id = f"{road_type.capitalize()}_{i}_to_{j}"
                # --- End TomTom Info Extraction ---

                # Append data for each point in the path to the main list
                for order, point in enumerate(decoded_path):
                    all_path_points_data.append({
                        'Path_ID': path_id,
                        'Road_Type': road_type,
                        'Point_Order': order + 1,
                        # Note: TomTom uses 'latitude' and 'longitude'
                        'Latitude': point['latitude'], 
                        'Longitude': point['longitude'],
                        'Congestion_Factor': round(congestion_factor, 2),
                        'Distance_Text': distance_text,
                        'Duration_in_Traffic_Text': duration_traffic_text,
                        'Duration_Value': duration_val # Base duration without traffic
                    })
        print(f"  Finished processing '{road_type}' paths.")

    except Exception as e:
        print(f"  An error occurred while processing '{road_type}': {e}")
        print("  This might happen if the road type doesn't exist in the specified area.")

# --- Save Combined Data to a Single CSV ---
print("-" * 50)
print("\nCombining data from all road types and saving to CSV...")
final__df = pd.DataFrame(all_path_points_data)
final__df.to_csv(OUTPUT_FILENAME, index=False)
print(f"\n Successfully generated combined traffic data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")


# Data For Mumbai

In [ ]:
import pandas as pd
from datetime import datetime
import osmnx as ox
import networkx as nx
import random
import math
import requests        
import urllib.parse    

# --- Configuration ---

API_KEY = "   " # change it with your TomTom API key


PLACE_LIST = [
    "Mumbai, India",
    "Navi Mumbai, India",
    "Thane, India",
    "Kalyan-Dombivli, India",
    "Vasai-Virar, India",
    "Mira-Bhayandar, India",
    "Panvel, India",
    "Ulhasnagar, India"
]

COMBINED_PLACE_LIST = "Mumbai"

POINTS_PER_ROAD_TYPE = 20 # Sample size per Road Type

RANDOM_STATE = 77

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{COMBINED_PLACE_LIST}_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Define the road types to process individually ---
ROAD_TYPES_TO_PROCESS = ['motorway', 'trunk', 'primary', 'secondary','tertiary']

# --- Helper functions to format TomTom's output (meters/seconds) ---

def format_duration(seconds):
    """Converts seconds into a human-readable 'X hours Y mins' or 'Y mins' string."""
    if seconds is None:
        return "N/A"
    minutes = math.ceil(seconds / 60)
    if minutes < 60:
        return f"{minutes} mins"
    else:
        hours = minutes // 60
        rem_minutes = minutes % 60
        return f"{hours} hours {rem_minutes} mins"

def format_distance(meters):
    """Converts meters into a human-readable 'X.Y km' string."""
    if meters is None:
        return "N/A"
    km = round(meters / 1000, 1)
    return f"{km} km"

# --- No TomTom Client Initialization needed ---

# This list will hold data from all road type loops
all_path_points_data = []

print(f"Starting traffic analysis for {PLACE_LIST}...")
print("-" * 50)

# --- Main loop to process each road type separately ---
for road_type in ROAD_TYPES_TO_PROCESS:
    print(f"\nProcessing road type: '{road_type.upper()}'")

    graphs_for_this_road_type = []
    print(f"  Downloading '{road_type}' network from OpenStreetMap for {len(PLACE_LIST)} places...")
    for place in PLACE_LIST:
        try:
            # Fetch network for ONLY the current road type
            road_filter = f'["highway"~"{road_type}"]'
            G_place = ox.graph_from_place(place, network_type='drive', custom_filter=road_filter)
            
            if G_place.nodes:
                graphs_for_this_road_type.append(G_place)
                print(f"    - Successfully fetched '{road_type}' for {place}")
            else:
                print(f"    - No '{road_type}' roads found in {place}.")
                
        except Exception as e:
            print(f"    - Error fetching data for {place}: {e}")
    
    if not graphs_for_this_road_type:
        print(f"  - No '{road_type}' roads found in any specified location. Skipping.")
        continue
        
    print(f"  Combining {len(graphs_for_this_road_type)} graphs for '{road_type}'...")
    G = nx.compose_all(graphs_for_this_road_type)

    try:
        if not G.nodes:
            print(f"  - Combined graph for '{road_type}' is empty. Skipping.")
            continue

        # Get all points for this specific network
        nodes = list(G.nodes)
        all_road_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in nodes]
        print(f"  Found {len(all_road_points)} total points for this road type.")

        if len(all_road_points) < 2:
            print(f"  - Not enough points ({len(all_road_points)}) to create paths. Skipping.")
            continue

        # Take a random sample of these points
        random.seed(RANDOM_STATE)
        sample_size = min(POINTS_PER_ROAD_TYPE, len(all_road_points))
        sampled_points = random.sample(all_road_points, sample_size)
        print(f"  Using a random sample of {len(sampled_points)} points.")

        # Generate paths and get traffic data for this sample
        total_paths = len(sampled_points) * (len(sampled_points) - 1)
        path_counter = 0

        if total_paths == 0:
            print("  - Only one point sampled, cannot create paths. Skipping.")
            continue

        print(f"  Starting TomTom API calls for {total_paths} paths...")
        for i, origin in enumerate(sampled_points):
            for j, destination in enumerate(sampled_points):
                if i == j:
                    continue

                path_counter += 1
                
                # --- TomTom API Call using 'requests' ---
                try:
                    origin_str = f"{origin[0]},{origin[1]}"
                    dest_str = f"{destination[0]},{destination[1]}"
                    
                    locations = urllib.parse.quote(f"{origin_str}:{dest_str}")
                    
                    depart_at_iso = datetime.now().isoformat()

                    base_url = f"https://api.tomtom.com/routing/1/calculateRoute/{locations}/json"
                    
                    params = {
                        'key': API_KEY,
                        'departAt': depart_at_iso,
                        'traffic': 'true',
                        'computeTravelTimeFor': 'all', # Request both traffic and no-traffic times
                        'travelMode': 'car',
                        'routeType': 'fastest'
                    }

                    # Make the GET request
                    response = requests.get(base_url, params=params)
                    response.raise_for_status() # Raise an exception for bad status codes (4xx, 5xx)
                    
                    directions_result = response.json()


                except requests.exceptions.RequestException as e:
                    print(f"      - TomTom API Error for path {i}-to-{j}: {e}")
                    continue 

                if not directions_result or 'routes' not in directions_result or not directions_result['routes']:
                    print(f"      - No route found by TomTom for path {i}-to-{j}.")
                    continue
                # --- End TomTom API Call ---

                # --- Extract TomTom Info ---
                route = directions_result['routes'][0]
                summary = route['summary']

                duration_val = summary.get('noTrafficTravelTimeInSeconds')
                duration_traffic_val = summary.get('travelTimeInSeconds')
                distance_val_meters = summary.get('lengthInMeters')
                
                # Handle cases where one value might be missing
                if duration_val is None:
                    duration_val = duration_traffic_val
                if duration_traffic_val is None:
                    duration_traffic_val = duration_val
                if duration_val is None or duration_traffic_val is None:
                    print(f"      - Missing duration data for path {i}-to-{j}. Skipping.")
                    continue
                
                # Calculate congestion
                congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

                leg = route['legs'][0]
                decoded_path = leg['points'] 

                # Get text representations using our helper functions
                distance_text = format_distance(distance_val_meters)
                duration_traffic_text = format_duration(duration_traffic_val)
                
                path_id = f"{road_type.capitalize()}_{i}_to_{j}"
                # --- End TomTom Info Extraction ---

                # Append data for each point in the path to the main list
                for order, point in enumerate(decoded_path):
                    all_path_points_data.append({
                        'Path_ID': path_id,
                        'Road_Type': road_type,
                        'Point_Order': order + 1,
                        # Note: TomTom uses 'latitude' and 'longitude'
                        'Latitude': point['latitude'], 
                        'Longitude': point['longitude'],
                        'Congestion_Factor': round(congestion_factor, 2),
                        'Distance_Text': distance_text,
                        'Duration_in_Traffic_Text': duration_traffic_text,
                        'Duration_Value': duration_val # Base duration without traffic
                    })
        print(f"  Finished processing '{road_type}' paths.")

    except Exception as e:
        print(f"  An error occurred while processing '{road_type}': {e}")
        print("  This might happen if the road type doesn't exist in the specified area.")

# --- Save Combined Data to a Single CSV ---
print("-" * 50)
print("\nCombining data from all road types and saving to CSV...")
final__df = pd.DataFrame(all_path_points_data)
final__df.to_csv(OUTPUT_FILENAME, index=False)
print(f"\n Successfully generated combined traffic data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")


Starting traffic analysis for ['Mumbai, India', 'Navi Mumbai, India', 'Thane, India', 'Kalyan-Dombivli, India', 'Vasai-Virar, India', 'Mira-Bhayandar, India', 'Panvel, India', 'Ulhasnagar, India']...
--------------------------------------------------

Processing road type: 'MOTORWAY'
    - Successfully fetched 'motorway' for Mumbai, India
    - Successfully fetched 'motorway' for Navi Mumbai, India
    - Successfully fetched 'motorway' for Thane, India
    - Error fetching data for Kalyan-Dombivli, India: No data elements in server response. Check query location/filters and log.
    - Error fetching data for Vasai-Virar, India: No data elements in server response. Check query location/filters and log.
    - Error fetching data for Mira-Bhayandar, India: Nominatim did not geocode query 'Mira-Bhayandar, India' to a geometry of type (Multi)Polygon.
    - Error fetching data for Panvel, India: No data elements in server response. Check query location/filters and log.
    - Error fetching d

# Data for Banglore

In [ ]:
import pandas as pd
from datetime import datetime
import osmnx as ox
import networkx as nx
import random
import math
import requests        
import urllib.parse    

# --- Configuration ---

API_KEY = "   " # change it with your TomTom API key

PLACE_LIST = [
    "Bengaluru, India",
    "Hoskote, India",
    "Devanahalli, India",
    "Doddaballapura, India",
    "Nelamangala, India",
    "Magadi, India",
    "Ramanagara, India",
    "Kanakapura, India",
    "Anekal, India",
    "Sarjapur, India",
    "Bidadi, India"
]


COMBINED_PLACE_LIST = "Mumbai"

POINTS_PER_ROAD_TYPE = 20 # Sample size per Road Type

RANDOM_STATE = 77

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{COMBINED_PLACE_LIST}_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Define the road types to process individually ---
ROAD_TYPES_TO_PROCESS = ['motorway', 'trunk', 'primary', 'secondary','tertiary']

# --- Helper functions to format TomTom's output (meters/seconds) ---

def format_duration(seconds):
    """Converts seconds into a human-readable 'X hours Y mins' or 'Y mins' string."""
    if seconds is None:
        return "N/A"
    minutes = math.ceil(seconds / 60)
    if minutes < 60:
        return f"{minutes} mins"
    else:
        hours = minutes // 60
        rem_minutes = minutes % 60
        return f"{hours} hours {rem_minutes} mins"

def format_distance(meters):
    """Converts meters into a human-readable 'X.Y km' string."""
    if meters is None:
        return "N/A"
    km = round(meters / 1000, 1)
    return f"{km} km"

# --- No TomTom Client Initialization needed ---

# This list will hold data from all road type loops
all_path_points_data = []

print(f"Starting traffic analysis for {PLACE_LIST}...")
print("-" * 50)

# --- Main loop to process each road type separately ---
for road_type in ROAD_TYPES_TO_PROCESS:
    print(f"\nProcessing road type: '{road_type.upper()}'")

    graphs_for_this_road_type = []
    print(f"  Downloading '{road_type}' network from OpenStreetMap for {len(PLACE_LIST)} places...")
    for place in PLACE_LIST:
        try:
            # Fetch network for ONLY the current road type
            road_filter = f'["highway"~"{road_type}"]'
            G_place = ox.graph_from_place(place, network_type='drive', custom_filter=road_filter)
            
            if G_place.nodes:
                graphs_for_this_road_type.append(G_place)
                print(f"    - Successfully fetched '{road_type}' for {place}")
            else:
                print(f"    - No '{road_type}' roads found in {place}.")
                
        except Exception as e:
            print(f"    - Error fetching data for {place}: {e}")
    
    if not graphs_for_this_road_type:
        print(f"  - No '{road_type}' roads found in any specified location. Skipping.")
        continue
        
    print(f"  Combining {len(graphs_for_this_road_type)} graphs for '{road_type}'...")
    G = nx.compose_all(graphs_for_this_road_type)

    try:
        if not G.nodes:
            print(f"  - Combined graph for '{road_type}' is empty. Skipping.")
            continue

        # Get all points for this specific network
        nodes = list(G.nodes)
        all_road_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in nodes]
        print(f"  Found {len(all_road_points)} total points for this road type.")

        if len(all_road_points) < 2:
            print(f"  - Not enough points ({len(all_road_points)}) to create paths. Skipping.")
            continue

        # Take a random sample of these points
        random.seed(RANDOM_STATE)
        sample_size = min(POINTS_PER_ROAD_TYPE, len(all_road_points))
        sampled_points = random.sample(all_road_points, sample_size)
        print(f"  Using a random sample of {len(sampled_points)} points.")

        # Generate paths and get traffic data for this sample
        total_paths = len(sampled_points) * (len(sampled_points) - 1)
        path_counter = 0

        if total_paths == 0:
            print("  - Only one point sampled, cannot create paths. Skipping.")
            continue

        print(f"  Starting TomTom API calls for {total_paths} paths...")
        for i, origin in enumerate(sampled_points):
            for j, destination in enumerate(sampled_points):
                if i == j:
                    continue

                path_counter += 1
                
                # --- TomTom API Call using 'requests' ---
                try:
                    origin_str = f"{origin[0]},{origin[1]}"
                    dest_str = f"{destination[0]},{destination[1]}"
                    
                    locations = urllib.parse.quote(f"{origin_str}:{dest_str}")
                    
                    depart_at_iso = datetime.now().isoformat()

                    base_url = f"https://api.tomtom.com/routing/1/calculateRoute/{locations}/json"
                    
                    params = {
                        'key': API_KEY,
                        'departAt': depart_at_iso,
                        'traffic': 'true',
                        'computeTravelTimeFor': 'all', # Request both traffic and no-traffic times
                        'travelMode': 'car',
                        'routeType': 'fastest'
                    }

                    # Make the GET request
                    response = requests.get(base_url, params=params)
                    response.raise_for_status() # Raise an exception for bad status codes (4xx, 5xx)
                    
                    directions_result = response.json()


                except requests.exceptions.RequestException as e:
                    print(f"      - TomTom API Error for path {i}-to-{j}: {e}")
                    continue 

                if not directions_result or 'routes' not in directions_result or not directions_result['routes']:
                    print(f"      - No route found by TomTom for path {i}-to-{j}.")
                    continue
                # --- End TomTom API Call ---

                # --- Extract TomTom Info ---
                route = directions_result['routes'][0]
                summary = route['summary']

                duration_val = summary.get('noTrafficTravelTimeInSeconds')
                duration_traffic_val = summary.get('travelTimeInSeconds')
                distance_val_meters = summary.get('lengthInMeters')
                
                # Handle cases where one value might be missing
                if duration_val is None:
                    duration_val = duration_traffic_val
                if duration_traffic_val is None:
                    duration_traffic_val = duration_val
                if duration_val is None or duration_traffic_val is None:
                    print(f"      - Missing duration data for path {i}-to-{j}. Skipping.")
                    continue
                
                # Calculate congestion
                congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

                leg = route['legs'][0]
                decoded_path = leg['points'] 

                # Get text representations using our helper functions
                distance_text = format_distance(distance_val_meters)
                duration_traffic_text = format_duration(duration_traffic_val)
                
                path_id = f"{road_type.capitalize()}_{i}_to_{j}"
                # --- End TomTom Info Extraction ---

                # Append data for each point in the path to the main list
                for order, point in enumerate(decoded_path):
                    all_path_points_data.append({
                        'Path_ID': path_id,
                        'Road_Type': road_type,
                        'Point_Order': order + 1,
                        # Note: TomTom uses 'latitude' and 'longitude'
                        'Latitude': point['latitude'], 
                        'Longitude': point['longitude'],
                        'Congestion_Factor': round(congestion_factor, 2),
                        'Distance_Text': distance_text,
                        'Duration_in_Traffic_Text': duration_traffic_text,
                        'Duration_Value': duration_val # Base duration without traffic
                    })
        print(f"  Finished processing '{road_type}' paths.")

    except Exception as e:
        print(f"  An error occurred while processing '{road_type}': {e}")
        print("  This might happen if the road type doesn't exist in the specified area.")

# --- Save Combined Data to a Single CSV ---
print("-" * 50)
print("\nCombining data from all road types and saving to CSV...")
final__df = pd.DataFrame(all_path_points_data)
final__df.to_csv(OUTPUT_FILENAME, index=False)
print(f"\n Successfully generated combined traffic data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")


# Kolkata data


In [ ]:
import pandas as pd
from datetime import datetime
import osmnx as ox
import networkx as nx
import random
import math
import requests        
import urllib.parse    

# --- Configuration ---

API_KEY = " " # change it with your TomTom API key

PLACE_LIST = [
    "Kolkata, India",
    "Howrah, India",
    "Bidhannagar,Kolkata, India",
    "Rajpur Sonarpur, India",
    "Maheshtala, India",
    "Serampore, India"
]

COMBINED_PLACE_LIST = "Kolkata"

POINTS_PER_ROAD_TYPE = 20 # Sample size per Road Type

RANDOM_STATE = 77

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{COMBINED_PLACE_LIST}_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Define the road types to process individually ---
ROAD_TYPES_TO_PROCESS = ['motorway', 'trunk', 'primary', 'secondary','tertiary']

# --- Helper functions to format TomTom's output (meters/seconds) ---

def format_duration(seconds):
    """Converts seconds into a human-readable 'X hours Y mins' or 'Y mins' string."""
    if seconds is None:
        return "N/A"
    minutes = math.ceil(seconds / 60)
    if minutes < 60:
        return f"{minutes} mins"
    else:
        hours = minutes // 60
        rem_minutes = minutes % 60
        return f"{hours} hours {rem_minutes} mins"

def format_distance(meters):
    """Converts meters into a human-readable 'X.Y km' string."""
    if meters is None:
        return "N/A"
    km = round(meters / 1000, 1)
    return f"{km} km"

# --- No TomTom Client Initialization needed ---

# This list will hold data from all road type loops
all_path_points_data = []

print(f"Starting traffic analysis for {PLACE_LIST}...")
print("-" * 50)

# --- Main loop to process each road type separately ---
for road_type in ROAD_TYPES_TO_PROCESS:
    print(f"\nProcessing road type: '{road_type.upper()}'")

    graphs_for_this_road_type = []
    print(f"  Downloading '{road_type}' network from OpenStreetMap for {len(PLACE_LIST)} places...")
    for place in PLACE_LIST:
        try:
            # Fetch network for ONLY the current road type
            road_filter = f'["highway"~"{road_type}"]'
            G_place = ox.graph_from_place(place, network_type='drive', custom_filter=road_filter)
            
            if G_place.nodes:
                graphs_for_this_road_type.append(G_place)
                print(f"    - Successfully fetched '{road_type}' for {place}")
            else:
                print(f"    - No '{road_type}' roads found in {place}.")
                
        except Exception as e:
            print(f"    - Error fetching data for {place}: {e}")
    
    if not graphs_for_this_road_type:
        print(f"  - No '{road_type}' roads found in any specified location. Skipping.")
        continue
        
    print(f"  Combining {len(graphs_for_this_road_type)} graphs for '{road_type}'...")
    G = nx.compose_all(graphs_for_this_road_type)

    try:
        if not G.nodes:
            print(f"  - Combined graph for '{road_type}' is empty. Skipping.")
            continue

        # Get all points for this specific network
        nodes = list(G.nodes)
        all_road_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in nodes]
        print(f"  Found {len(all_road_points)} total points for this road type.")

        if len(all_road_points) < 2:
            print(f"  - Not enough points ({len(all_road_points)}) to create paths. Skipping.")
            continue

        # Take a random sample of these points
        random.seed(RANDOM_STATE)
        sample_size = min(POINTS_PER_ROAD_TYPE, len(all_road_points))
        sampled_points = random.sample(all_road_points, sample_size)
        print(f"  Using a random sample of {len(sampled_points)} points.")

        # Generate paths and get traffic data for this sample
        total_paths = len(sampled_points) * (len(sampled_points) - 1)
        path_counter = 0

        if total_paths == 0:
            print("  - Only one point sampled, cannot create paths. Skipping.")
            continue

        print(f"  Starting TomTom API calls for {total_paths} paths...")
        for i, origin in enumerate(sampled_points):
            for j, destination in enumerate(sampled_points):
                if i == j:
                    continue

                path_counter += 1
                
                # --- TomTom API Call using 'requests' ---
                try:
                    # TomTom uses (lat, lon)
                    origin_str = f"{origin[0]},{origin[1]}"
                    dest_str = f"{destination[0]},{destination[1]}"
                    
                    # URL-encode the location string "lat,lon:lat,lon"
                    locations = urllib.parse.quote(f"{origin_str}:{dest_str}")
                    
                    # Get current time in ISO format for departure time
                    depart_at_iso = datetime.now().isoformat()

                    # Construct the API URL
                    base_url = f"https://api.tomtom.com/routing/1/calculateRoute/{locations}/json"
                    
                    params = {
                        'key': API_KEY,
                        'departAt': depart_at_iso,
                        'traffic': 'true',
                        'computeTravelTimeFor': 'all', # Request both traffic and no-traffic times
                        'travelMode': 'car',
                        'routeType': 'fastest'
                    }

                    # Make the GET request
                    response = requests.get(base_url, params=params)
                    response.raise_for_status() # Raise an exception for bad status codes (4xx, 5xx)
                    
                    directions_result = response.json()


                except requests.exceptions.RequestException as e:
                    print(f"      - TomTom API Error for path {i}-to-{j}: {e}")
                    continue # Skip this path

                if not directions_result or 'routes' not in directions_result or not directions_result['routes']:
                    print(f"      - No route found by TomTom for path {i}-to-{j}.")
                    continue
                # --- End TomTom API Call ---

                # --- Extract TomTom Info ---
                route = directions_result['routes'][0]
                summary = route['summary']

                # TomTom provides no-traffic and with-traffic durations
                duration_val = summary.get('noTrafficTravelTimeInSeconds')
                duration_traffic_val = summary.get('travelTimeInSeconds')
                distance_val_meters = summary.get('lengthInMeters')
                
                # Handle cases where one value might be missing
                if duration_val is None:
                    duration_val = duration_traffic_val
                if duration_traffic_val is None:
                    duration_traffic_val = duration_val
                if duration_val is None or duration_traffic_val is None:
                    print(f"      - Missing duration data for path {i}-to-{j}. Skipping.")
                    continue
                
                # Calculate congestion
                congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

                # TomTom provides decoded points directly in legs
                leg = route['legs'][0]
                decoded_path = leg['points'] # This is a list of {'latitude': y, 'longitude': x}

                # Get text representations using our helper functions
                distance_text = format_distance(distance_val_meters)
                duration_traffic_text = format_duration(duration_traffic_val)
                
                path_id = f"{road_type.capitalize()}_{i}_to_{j}"
                # --- End TomTom Info Extraction ---

                # Append data for each point in the path to the main list
                for order, point in enumerate(decoded_path):
                    all_path_points_data.append({
                        'Path_ID': path_id,
                        'Road_Type': road_type,
                        'Point_Order': order + 1,
                        # Note: TomTom uses 'latitude' and 'longitude'
                        'Latitude': point['latitude'], 
                        'Longitude': point['longitude'],
                        'Congestion_Factor': round(congestion_factor, 2),
                        'Distance_Text': distance_text,
                        'Duration_in_Traffic_Text': duration_traffic_text,
                        'Duration_Value': duration_val # Base duration without traffic
                    })
        print(f"  Finished processing '{road_type}' paths.")

    except Exception as e:
        print(f"  An error occurred while processing '{road_type}': {e}")
        print("  This might happen if the road type doesn't exist in the specified area.")

# --- Save Combined Data to a Single CSV ---
print("-" * 50)
print("\nCombining data from all road types and saving to CSV...")
final__df = pd.DataFrame(all_path_points_data)
final__df.to_csv(OUTPUT_FILENAME, index=False)
print(f"\n Successfully generated combined traffic data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")


c:\conda\envs\pyenv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Starting traffic analysis for ['Kolkata, India', 'Howrah, India', 'Bidhannagar, India', 'New Town, Kolkata, India', 'Rajpur Sonarpur, India', 'Maheshtala, India', 'Baranagar, India', 'Dum Dum, India', 'Serampore, India']...
--------------------------------------------------

Processing road type: 'MOTORWAY'
    - Successfully fetched 'motorway' for Kolkata, India
  Combining 1 graphs for 'motorway'...
  Found 8 total points for this road type.
  Using a random sample of 8 points.
  Starting Google Maps API calls for 56 paths...
  Finished processing 'motorway' paths.
    - Successfully fetched 'motorway' for Howrah, India
  Combining 2 graphs for 'motorway'...
  Found 14 total points for this road type.
  Using a random sample of 14 points.
  Starting Google Maps API calls for 182 paths...
  Finished processing 'motorway' paths.
    - Error fetching data for Bidhannagar, India: No data elements in server response. Check query location/filters and log.
  Combining 2 graphs for 'motorway

# Using TOM TOM API

In [2]:
!pip install tomtom


  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ------------------------------ --------- 0.8/1.0 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.1 MB/s  0:00:00
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.8 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.8 MB 2.6 MB/s eta 0:00:04
   ------ --------------------------------- 1.6/9.8 MB 2.8 MB/s eta 0:00:03
   --------- ------------------------------ 2.4/9.8 MB 3.0 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/9.8 MB 3.1 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.8 MB 3.5 MB/s eta 0:00:02
   --------------------- ------------------ 5.2/9.8 MB 3.9 MB/s eta 0:00:02
   -----

In [ ]:
import pandas as pd
from datetime import datetime
import osmnx as ox
import networkx as nx
import random
import math
import requests        
import urllib.parse    

# --- Configuration ---

API_KEY = " " # change it with your TomTom API key

PLACE_LIST = [
    "Kolkata, India",
    "Howrah, India",
    "Bidhannagar,Kolkata, India",
    "Rajpur Sonarpur, India",
    "Maheshtala, India",
    "Serampore, India"
]

COMBINED_PLACE_LIST = "Kolkata"

POINTS_PER_ROAD_TYPE = 20 # Sample size per Road Type

RANDOM_STATE = 77

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"traffic_report_{COMBINED_PLACE_LIST}_{timestamp}.csv"
OUTPUT_FILENAME = filename

# --- Define the road types to process individually ---
ROAD_TYPES_TO_PROCESS = ['motorway', 'trunk', 'primary', 'secondary','tertiary']

# --- Helper functions to format TomTom's output (meters/seconds) ---

def format_duration(seconds):
    """Converts seconds into a human-readable 'X hours Y mins' or 'Y mins' string."""
    if seconds is None:
        return "N/A"
    minutes = math.ceil(seconds / 60)
    if minutes < 60:
        return f"{minutes} mins"
    else:
        hours = minutes // 60
        rem_minutes = minutes % 60
        return f"{hours} hours {rem_minutes} mins"

def format_distance(meters):
    """Converts meters into a human-readable 'X.Y km' string."""
    if meters is None:
        return "N/A"
    km = round(meters / 1000, 1)
    return f"{km} km"

# --- No TomTom Client Initialization needed ---

# This list will hold data from all road type loops
all_path_points_data = []

print(f"Starting traffic analysis for {PLACE_LIST}...")
print("-" * 50)

# --- Main loop to process each road type separately ---
for road_type in ROAD_TYPES_TO_PROCESS:
    print(f"\nProcessing road type: '{road_type.upper()}'")

    graphs_for_this_road_type = []
    print(f"  Downloading '{road_type}' network from OpenStreetMap for {len(PLACE_LIST)} places...")
    for place in PLACE_LIST:
        try:
            # Fetch network for ONLY the current road type
            road_filter = f'["highway"~"{road_type}"]'
            G_place = ox.graph_from_place(place, network_type='drive', custom_filter=road_filter)
            
            if G_place.nodes:
                graphs_for_this_road_type.append(G_place)
                print(f"    - Successfully fetched '{road_type}' for {place}")
            else:
                print(f"    - No '{road_type}' roads found in {place}.")
                
        except Exception as e:
            print(f"    - Error fetching data for {place}: {e}")
    
    if not graphs_for_this_road_type:
        print(f"  - No '{road_type}' roads found in any specified location. Skipping.")
        continue
        
    print(f"  Combining {len(graphs_for_this_road_type)} graphs for '{road_type}'...")
    G = nx.compose_all(graphs_for_this_road_type)

    try:
        if not G.nodes:
            print(f"  - Combined graph for '{road_type}' is empty. Skipping.")
            continue

        # Get all points for this specific network
        nodes = list(G.nodes)
        all_road_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in nodes]
        print(f"  Found {len(all_road_points)} total points for this road type.")

        if len(all_road_points) < 2:
            print(f"  - Not enough points ({len(all_road_points)}) to create paths. Skipping.")
            continue

        # Take a random sample of these points
        random.seed(RANDOM_STATE)
        sample_size = min(POINTS_PER_ROAD_TYPE, len(all_road_points))
        sampled_points = random.sample(all_road_points, sample_size)
        print(f"  Using a random sample of {len(sampled_points)} points.")

        # Generate paths and get traffic data for this sample
        total_paths = len(sampled_points) * (len(sampled_points) - 1)
        path_counter = 0

        if total_paths == 0:
            print("  - Only one point sampled, cannot create paths. Skipping.")
            continue

        print(f"  Starting TomTom API calls for {total_paths} paths...")
        for i, origin in enumerate(sampled_points):
            for j, destination in enumerate(sampled_points):
                if i == j:
                    continue

                path_counter += 1
                
                # --- TomTom API Call using 'requests' ---
                try:
                    # TomTom uses (lat, lon)
                    origin_str = f"{origin[0]},{origin[1]}"
                    dest_str = f"{destination[0]},{destination[1]}"
                    
                    # URL-encode the location string "lat,lon:lat,lon"
                    locations = urllib.parse.quote(f"{origin_str}:{dest_str}")
                    
                    # Get current time in ISO format for departure time
                    depart_at_iso = datetime.now().isoformat()

                    # Construct the API URL
                    base_url = f"https://api.tomtom.com/routing/1/calculateRoute/{locations}/json"
                    
                    params = {
                        'key': API_KEY,
                        'departAt': depart_at_iso,
                        'traffic': 'true',
                        'computeTravelTimeFor': 'all', # Request both traffic and no-traffic times
                        'travelMode': 'car',
                        'routeType': 'fastest'
                    }

                    # Make the GET request
                    response = requests.get(base_url, params=params)
                    response.raise_for_status() # Raise an exception for bad status codes (4xx, 5xx)
                    
                    directions_result = response.json()


                except requests.exceptions.RequestException as e:
                    print(f"      - TomTom API Error for path {i}-to-{j}: {e}")
                    continue # Skip this path

                if not directions_result or 'routes' not in directions_result or not directions_result['routes']:
                    print(f"      - No route found by TomTom for path {i}-to-{j}.")
                    continue
                # --- End TomTom API Call ---

                # --- Extract TomTom Info ---
                route = directions_result['routes'][0]
                summary = route['summary']

                # TomTom provides no-traffic and with-traffic durations
                duration_val = summary.get('noTrafficTravelTimeInSeconds')
                duration_traffic_val = summary.get('travelTimeInSeconds')
                distance_val_meters = summary.get('lengthInMeters')
                
                # Handle cases where one value might be missing
                if duration_val is None:
                    duration_val = duration_traffic_val
                if duration_traffic_val is None:
                    duration_traffic_val = duration_val
                if duration_val is None or duration_traffic_val is None:
                    print(f"      - Missing duration data for path {i}-to-{j}. Skipping.")
                    continue
                
                # Calculate congestion
                congestion_factor = duration_traffic_val / duration_val if duration_val > 0 else 1

                # TomTom provides decoded points directly in legs
                leg = route['legs'][0]
                decoded_path = leg['points'] # This is a list of {'latitude': y, 'longitude': x}

                # Get text representations using our helper functions
                distance_text = format_distance(distance_val_meters)
                duration_traffic_text = format_duration(duration_traffic_val)
                
                path_id = f"{road_type.capitalize()}_{i}_to_{j}"
                # --- End TomTom Info Extraction ---

                # Append data for each point in the path to the main list
                for order, point in enumerate(decoded_path):
                    all_path_points_data.append({
                        'Path_ID': path_id,
                        'Road_Type': road_type,
                        'Point_Order': order + 1,
                        # Note: TomTom uses 'latitude' and 'longitude'
                        'Latitude': point['latitude'], 
                        'Longitude': point['longitude'],
                        'Congestion_Factor': round(congestion_factor, 2),
                        'Distance_Text': distance_text,
                        'Duration_in_Traffic_Text': duration_traffic_text,
                        'Duration_Value': duration_val # Base duration without traffic
                    })
        print(f"  Finished processing '{road_type}' paths.")

    except Exception as e:
        print(f"  An error occurred while processing '{road_type}': {e}")
        print("  This might happen if the road type doesn't exist in the specified area.")

# --- Save Combined Data to a Single CSV ---
print("-" * 50)
print("\nCombining data from all road types and saving to CSV...")
final__df = pd.DataFrame(all_path_points_data)
final__df.to_csv(OUTPUT_FILENAME, index=False)
print(f"\n Successfully generated combined traffic data!")
print(f"'{OUTPUT_FILENAME}' is now ready for Tableau.")


c:\conda\envs\pyenv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Starting traffic analysis for ['Kolkata, India', 'Howrah, India', 'Bidhannagar,Kolkata, India', 'Rajpur Sonarpur, India', 'Maheshtala, India', 'Serampore, India']...
--------------------------------------------------

Processing road type: 'MOTORWAY'
    - Successfully fetched 'motorway' for Kolkata, India
    - Successfully fetched 'motorway' for Howrah, India
    - Error fetching data for Bidhannagar,Kolkata, India: No data elements in server response. Check query location/filters and log.
    - Error fetching data for Rajpur Sonarpur, India: No data elements in server response. Check query location/filters and log.
    - Error fetching data for Maheshtala, India: No data elements in server response. Check query location/filters and log.
    - Error fetching data for Serampore, India: Nominatim did not geocode query 'Serampore, India' to a geometry of type (Multi)Polygon.
  Combining 2 graphs for 'motorway'...
  Found 14 total points for this road type.
  Using a random sample of 14 

: 

In [3]:
final__df

,Path_ID,Road_Type,Point_Order,Latitude,Longitude,Congestion_Factor,Distance_Text,Duration_in_Traffic_Text,Duration_Value
0,Motorway_0_to_1,motorway,1,22.56346,88.31833,1.06,12.9 km,22 mins,1201
1,Motorway_0_to_1,motorway,2,22.56351,88.31827,1.06,12.9 km,22 mins,1201
2,Motorway_0_to_1,motorway,3,22.56364,88.31807,1.06,12.9 km,22 mins,1201
3,Motorway_0_to_1,motorway,4,22.56383,88.31778,1.06,12.9 km,22 mins,1201
4,Motorway_0_to_1,motorway,5,22.56397,88.31791,1.06,12.9 km,22 mins,1201
...,...,...,...,...,...,...,...,...,...
338644,Tertiary_8_to_0,tertiary,436,22.44373,88.41757,1.00,16.5 km,35 mins,2078
338645,Tertiary_8_to_0,tertiary,437,22.44373,88.41761,1.00,16.5 km,35 mins,2078
338646,Tertiary_8_to_0,tertiary,438,22.44374,88.41797,1.00,16.5 km,35 mins,2078
338647,Tertiary_8_to_0,tertiary,439,22.44374,88.41801,1.00,16.5 km,35 mins,2078


In [2]:
final__df.to_csv(OUTPUT_FILENAME, index=False)


# Getting economic hubs of an country example

In [1]:
import requests
import pandas as pd
import time
import math

# --- Configuration ---
API_KEY = "Api key "
OUTPUT_FILENAME = "economic_hotspots.csv"

AREA_BOUNDS = {
    "sw": (28.341944, 76.811111),
    "ne": (28.928889, 77.473056)
}

ECONOMIC_CATEGORIES = {
    "Commercial": ["business park in Delhi NCR", "IT park in Gurgaon", "corporate office in Noida"],
    "Retail": ["shopping mall in Delhi NCR", "major market in Delhi"],
    "Industrial": ["industrial area in Delhi NCR", "factory in Faridabad"],
    "Transit Hub": ["major metro station in Delhi", "major railway station in Delhi"]
}


def get_area_center_and_radius(bounds):
    """Calculates the center and radius of the bounding box to focus the search."""
    center_lat = (bounds['sw'][0] + bounds['ne'][0]) / 2
    center_lon = (bounds['sw'][1] + bounds['ne'][1]) / 2
    R = 6371000
    lat1, lon1 = math.radians(center_lat), math.radians(center_lon)
    lat2, lon2 = math.radians(bounds['ne'][0]), math.radians(bounds['ne'][1])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    radius = R * c
    return ((center_lat, center_lon), radius)


def search_places_new_api(query, location, radius, api_key):
    """Uses the new Places API (v1) text search."""
    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.displayName,places.id,places.formattedAddress,places.location"
    }
    payload = {
        "textQuery": query,
        "locationBias": {
            "circle": {
                "center": {"latitude": location[0], "longitude": location[1]},
                "radius": radius
            }
        }
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code != 200:
        print(f"Places API Error ({response.status_code}): {response.text}")
        return []
    data = response.json()
    return data.get("places", [])


def collect_hotspots():
    print("--- Starting Economic Hotspot Collection ---")

    search_center, search_radius = get_area_center_and_radius(AREA_BOUNDS)
    unique_places = {}

    for category, queries in ECONOMIC_CATEGORIES.items():
        print(f"\nSearching for category: '{category}'...")
        for query in queries:
            print(f"  Using query: '{query}'")
            try:
                results = search_places_new_api(query, search_center, search_radius, API_KEY)
                for place in results:
                    place_id = place.get("id")
                    if place_id not in unique_places:
                        loc = place["location"]
                        unique_places[place_id] = {
                            "Name": place["displayName"]["text"],
                            "Category": category,
                            "Latitude": loc["latitude"],
                            "Longitude": loc["longitude"],
                            "Address": place.get("formattedAddress", "N/A")
                        }
                time.sleep(1)
            except Exception as e:
                print(f"    -> Error during search for '{query}': {e}")

    if not unique_places:
        print("No hotspots were found. Check API key or quota limits.")
        return

    hotspots_df = pd.DataFrame(unique_places.values())
    hotspots_df.to_csv(OUTPUT_FILENAME, index=False)
    print(f"\n--- Collection Complete! ---")
    print(f"Saved {len(hotspots_df)} unique economic hotspots to '{OUTPUT_FILENAME}'")


if __name__ == "__main__":
    collect_hotspots()


--- Starting Economic Hotspot Collection ---

Searching for category: 'Commercial'...
  Using query: 'business park in Delhi NCR'
  Using query: 'IT park in Gurgaon'
  Using query: 'corporate office in Noida'

Searching for category: 'Retail'...
  Using query: 'shopping mall in Delhi NCR'
  Using query: 'major market in Delhi'

Searching for category: 'Industrial'...
  Using query: 'industrial area in Delhi NCR'
  Using query: 'factory in Faridabad'

Searching for category: 'Transit Hub'...
  Using query: 'major metro station in Delhi'
  Using query: 'major railway station in Delhi'

--- Collection Complete! ---
Saved 129 unique economic hotspots to 'economic_hotspots.csv'


In [2]:
economic_hotspots = pd.read_csv("economic_hotspots.csv")
economic_hotspots.head()

,Name,Category,Latitude,Longitude,Address
0,Best Business Park,Commercial,28.691897,77.147454,"plot no. P, 2, Netaji Subhash Place, Shakurpur..."
1,DLF Corporate Park,Commercial,28.482756,77.105140,"Moulsari Ave, Garden Estate, DLF Phase 3, Sect..."
2,Global Business Park,Commercial,28.481231,77.102032,"Mehrauli-Gurgaon Rd, Sikanderpur, Sector 26, G..."
3,Embassy Galaxy Business Park,Commercial,28.624444,77.370247,"A-44 & 45, Sushil Marg, Block A, Industrial Ar..."
4,NSIC Business park,Commercial,28.551444,77.265461,"Software Technology Park, FB-05, NSIC, NSIC Es..."
